In [1]:
import cv2
import numpy as np
from pythonRLSA import rlsa
import math
import pytesseract
from PIL import Image

In [17]:
image = cv2.imread('./../newsImages/20220308.jpg') #reading the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # converting to grayscale image
(thresh, im_bw) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # converting to binary image



In [18]:
cv2.imshow('First', im_bw)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
mask = np.ones(image.shape[:2], dtype="uint8") * 255

In [34]:
contours = cv2.findContours(~im_bw,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0] # find contours
areas = [cv2.contourArea(c) for c in contours if cv2.contourArea(c) != 0] # find area of contour
avgArea = sum(areas)/len(areas) # finding average area
for c in contours:# average area heuristics
    if cv2.contourArea(c)>10*avgArea:
        [x,y,w,h] = cv2.boundingRect(c)
        cv2.rectangle(mask, (x,y), (x+w, y+h), (0,0,0),-1)
        #cv2.drawContours(mask, [c], -1, 0, -1)
without_img = im_bw - ~mask
#binary = cv2.bitwise_and(im_bw, im_bw, mask=mask) # subtracting the noise

In [25]:
cv2.imshow('mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [36]:
cv2.imshow('without_img', without_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
print(avgArea)

182.40306391970418


In [ ]:
for contour in contours:
    """
    draw a rectangle around those contours on main image
    """
    [x,y,w,h] = cv2.boundingRect(contour)
    cv2.rectangle(image, (x,y), (x+w,y+h), (0, 255, 0), 1)

In [32]:
mask = np.ones(image.shape[:2], dtype="uint8") * 255 # create blank image of same dimension of the original image
contours = cv2.findContours(~binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
heights = [cv2.boundingRect(contour)[3] for contour in contours] # collecting heights of each contour
avgheight = sum(heights)/len(heights) # average height
print(avgheight)
# finding the larger text
for c in contours:
    [x,y,w,h] = cv2.boundingRect(c)
    if h > 6*avgheight:
        cv2.drawContours(mask, [c], -1, 0, -1)





4.449208575435785


In [ ]:
cv2.imshow('test', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
x, y = mask.shape # image dimensions

value = max(math.ceil(x/100),math.ceil(y/100))+20
mask = rlsa.rlsa(mask, True, False, value) #rlsa application

cv2.imshow('mask1', mask)

contours = cv2.findContours(~mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]

mask2 = np.ones(image.shape, dtype="uint8") * 255 # blank 3 layer image
for contour in contours:
    [x, y, w, h] = cv2.boundingRect(contour)
    if w > 0.60*image.shape[1]:# width heuristic applied
        title = image[y: y+h, x: x+w] 
        mask2[y: y+h, x: x+w] = title # copied title contour onto the blank image
        image[y: y+h, x: x+w] = 255 # nullified the title contour on original image

title = pytesseract.image_to_string(Image.fromarray(mask2))
content = pytesseract.image_to_string(Image.fromarray(image))

print('title - {0}, content - {1}'.format(title, content))

In [6]:
cv2.imshow('mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
cv2.destroyAllWindows()